<a href="https://colab.research.google.com/github/gtoubian/cce/blob/main/3_5_PySpark_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Warmup
Import the numpy, pandas, matplotlib and pyspark packages in the cell below.

In [130]:
!pip install pyspark
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt

#Exercise
We are going to be using data on Student Performance during exams. Obtain the data from the following url and set up a Schema for it which you will load into pySpark. https://www.kaggle.com/spscientist/students-performance-in-exams. Set up the PySpark object and create a DataFrame on PySpark for the dataset.

In [131]:
from google.colab import files
data = files.upload()

Saving StudentsPerformance.csv to StudentsPerformance (9).csv


In [132]:
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

In [133]:
my_spark = SparkSession.builder.getOrCreate()

In [134]:
from pyspark.sql.types import *

data_schema = [
               StructField('gender', StringType(), True),
               StructField('race/ethnicity', StringType(), True),
               StructField('parental level of education', StringType(), True),
               StructField('lunch', StringType(), True),
               StructField('test preparation course', StringType(), True),
               StructField('math score', IntegerType(), True),
               StructField('reading score', IntegerType(), True),
               StructField('writing score', IntegerType(), True),

            ]

final_struc = StructType(fields = data_schema)

data = spark.read.csv('StudentsPerformance.csv',sep = ',', header = True, schema = final_struc)

In [135]:
data.show(10)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|gender|race/ethnicity|parental level of education|       lunch|test preparation course|math score|reading score|writing score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+
|female|       group B|          bachelor's degree|    standard|                   none|        72|           72|           74|
|female|       group C|               some college|    standard|              completed|        69|           90|           88|
|female|       group B|            master's degree|    standard|                   none|        90|           95|           93|
|  male|       group A|         associate's degree|free/reduced|                   none|        47|           57|           44|
|  male|       group C|               some college|    standard|                   none|        76|     

Get an average score for each student using their math, reading and writing scores. 

In [136]:
from pyspark.sql.functions import col

scorescolumns = [col('math score'), col('reading score'), col('writing score')]

averagefc = sum(x for x in scorescolumns)/len(scorescolumns)

data.withColumn('average score', averagefc).show(truncate=False)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------------------+
|gender|race/ethnicity|parental level of education|lunch       |test preparation course|math score|reading score|writing score|average score     |
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+------------------+
|female|group B       |bachelor's degree          |standard    |none                   |72        |72           |74           |72.66666666666667 |
|female|group C       |some college               |standard    |completed              |69        |90           |88           |82.33333333333333 |
|female|group B       |master's degree            |standard    |none                   |90        |95           |93           |92.66666666666667 |
|male  |group A       |associate's degree         |free/reduced|none                   |47        |57           |44   

Get a Count of the Number of Boys and girls in the class

In [137]:
data.groupBy('gender').count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  518|
|  male|  482|
+------+-----+



Apply the rank function over the overall average score that you had calculated previously.

In [138]:
data = data.withColumn('average score', averagefc)

In [139]:
from pyspark.sql import Window
windowSpec = Window.orderBy(desc('average score'))
import pyspark.sql.functions as f
from pyspark.sql.functions import desc
data.withColumn("average score", f.dense_rank().over(windowSpec)).show(10, truncate = False)

+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-------------+
|gender|race/ethnicity|parental level of education|lunch       |test preparation course|math score|reading score|writing score|average score|
+------+--------------+---------------------------+------------+-----------------------+----------+-------------+-------------+-------------+
|female|group E       |bachelor's degree          |standard    |none                   |100       |100          |100          |1            |
|male  |group E       |bachelor's degree          |standard    |completed              |100       |100          |100          |1            |
|female|group E       |associate's degree         |standard    |none                   |100       |100          |100          |1            |
|female|group E       |bachelor's degree          |standard    |completed              |99        |100          |100          |2            |
|femal

Do the same exercises above but using a Pandas DataFrame instead.

In [140]:
df = pd.read_csv ('StudentsPerformance.csv')

In [141]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75
...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95
996,male,group C,high school,free/reduced,none,62,55,55
997,female,group C,high school,free/reduced,completed,59,71,65
998,female,group D,some college,standard,completed,68,78,77


In [142]:
df['average score'] = df[['math score','reading score','writing score']].mean(axis=1)

In [143]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667
1,female,group C,some college,standard,completed,69,90,88,82.333333
2,female,group B,master's degree,standard,none,90,95,93,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333
4,male,group C,some college,standard,none,76,78,75,76.333333
...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,94.000000
996,male,group C,high school,free/reduced,none,62,55,55,57.333333
997,female,group C,high school,free/reduced,completed,59,71,65,65.000000
998,female,group D,some college,standard,completed,68,78,77,74.333333


In [144]:
df['gender'].value_counts()

female    518
male      482
Name: gender, dtype: int64

In [161]:
df["average rank"] = df["average score"].rank(method = 'dense', ascending=0) 

In [162]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score,average rank
0,female,group B,bachelor's degree,standard,none,72,72,74,72.666667,76.0
1,female,group C,some college,standard,completed,69,90,88,82.333333,47.0
2,female,group B,master's degree,standard,none,90,95,93,92.666667,16.0
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.333333,146.0
4,male,group C,some college,standard,none,76,78,75,76.333333,65.0
...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,94.000000,12.0
996,male,group C,high school,free/reduced,none,62,55,55,57.333333,122.0
997,female,group C,high school,free/reduced,completed,59,71,65,65.000000,99.0
998,female,group D,some college,standard,completed,68,78,77,74.333333,71.0


In [163]:
df.sort_values(by='average rank', ascending = True)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,average score,average rank
916,male,group E,bachelor's degree,standard,completed,100,100,100,100.000000,1.0
458,female,group E,bachelor's degree,standard,none,100,100,100,100.000000,1.0
962,female,group E,associate's degree,standard,none,100,100,100,100.000000,1.0
114,female,group E,bachelor's degree,standard,completed,99,100,100,99.666667,2.0
179,female,group D,some high school,standard,completed,97,100,100,99.000000,3.0
...,...,...,...,...,...,...,...,...,...,...
17,female,group B,some high school,free/reduced,none,18,32,28,26.000000,190.0
327,male,group A,some college,free/reduced,none,28,23,19,23.333333,191.0
596,male,group B,high school,free/reduced,none,30,24,15,23.000000,192.0
980,female,group B,high school,free/reduced,none,8,24,23,18.333333,193.0


#Python Practice Question
Concordia University has the following grading policy:

Every student receives a grade in the inclusive range from 0 to 100 . Any less than 40 is a failing grade.

Sam is a professor at the university and likes to round each student's grade according to these rules: If the difference between the grade and the next multiple of 5 is less than 3, round up to the next multiple of 5. If the value of the grade is less than 38, no rounding occurs as the result will still be a failing grade. For example, grade = 84 will be rounded to 85 but grade = 29 will not be rounded because the rounding would result in a number that is less than 40.

Given the initial value of grade for each of Sam's students, write code to automate the rounding process.
Examples



```
round to  (85 - 84 is less than 3)
 do not round (result is less than 40)
 do not round (60 - 57 is 3 or higher)
```




```
Sample Input
0
4
73
67
38
33

Sample Output 
0
4
75
67
40
33
```





In [164]:
def Students(grades):

    for x,y in enumerate(grades):
        if(y>=38) and (y%5)>=3:
            grades[x]=y+5-(y%5)
    return (grades)

grades = [0, 4, 73, 67, 38, 33]
Students(grades)

[0, 4, 75, 67, 40, 33]